In [ ]:
# Keepers

## Engage brain before starting engine!!!
## Make it simple - maximize work avoided.

## VS Code Shortcuts

- ctrl + =: reset editor group size (make window widths equal)
- cmd + k + s: show shortcuts

- cmd + `:   show/hide terminal
- ctrl + b:  show/hide panel
- shift + cmd + e: switch view to panel and back
- shift + cmd + f: switch to find
- cmd + k, z: zen mode
- shift + cmd + b: open command palette

- cmd + p: file explorer
- cmd + up/down: beginning/end of file
- shift + F12: show uses for cursor item
- F12: goto definition of cursor item
- **F2: rename all instances of cursor variable**
- opt + up/down: move line up/down
- shift + opt + up/down: copy line up/down

- opt + cmd + left/right: previous/next tab



## infinite recursion loop in master & provisional
--- 2024-08-08 08:01:00,035 - DEBUG - view.maybe_save
    calling chech_item_hsh
--- 2024-08-08 08:01:00,077 - DEBUG - model.check_item_hsh
    beginning check_item_hsh for maybe_save()
--- 2024-08-08 08:01:00,077 - DEBUG - view.maybe_save
    back from chech_item_hsh with []
--- 2024-08-08 08:01:00,077 - DEBUG - model.show_active_view
    using cache for do next
--- 2024-08-08 08:01:00,077 - DEBUG - common.wrapper
    ⏱ show_active_view (do next) took 0.0000 seconds
--- 2024-08-08 08:01:00,077 - DEBUG - common.wrapper
    ⏱ maybe_save took 0.0424 seconds
--- 2024-08-08 08:01:00,080 - DEBUG - model.update_item_hsh
    beginning update_item_hsh for item_changed()
--- 2024-08-08 08:01:00,081 - DEBUG - model.check_item_hsh
    beginning check_item_hsh for update_item_hsh()
--- 2024-08-08 08:01:00,083 - DEBUG - model.do_update
    beginning do_update for update_item_hsh()
--- 2024-08-08 08:01:00,132 - DEBUG - model.get_repetitions
    beginning Item get_repetitions for do_update()
--- 2024-08-08 08:01:00,133 - DEBUG - model.update_item_hsh
    beginning update_item_hsh for get_repetitions()


## Job Prerequisites

I’m thinking that a new, out of the box approach might be better than fixing the current approach. Instead of this:

   \- current prereqs approach

      @j alpha &i a
      @j beta &i b &p a
      @j gamma &i c &p b

what about this:

   \- proposed prereqs approach
   
      @j alpha
      @j beta &p -1
      @j gamma &p -1

Here the “&i” would be eliminated in favor of an implicit id based on the position in the list of jobs. Prerequisites would then be stated in relative terms so that “&p -1” makes the previous job a prerequisite for the current job. Then, adding a new element in which every job is still a prerequisite for the following job would be pretty simple:

   \- proposed prereqs approach

      @j alpha
      @j beta &p -1
      @j new &p -1
      @j gamma &p -1

With the proposed approach, no job needs an &i and each job that only requires the one before it just needs an “&p -1” entry.

Here is a more complicated example of the current and proposed systems:

   \- current prereqs

      @j alpha &i a
      @j beta &i b
      @j gamma &i c &p a, b

   \- proposed prereqs

      @j alpha 
      @j beta 
      @j gamma &p -1, -2

In both cases neither alpha or beta have prerequisites but both are prerequisites for gamma. With an additional job that also has no prerequisites and is also a prerequisite for gamma:

   \- current prereqs

      @j alpha &i a
      @j beta &i b
      @j new &i z 
      @j gamma &i c &p a, b, z

   \- proposed prereqs

      @j alpha 
      @j beta 
      @j new 
      @j gamma &p -1, -2, -3

I’m thinking this would be simpler and better. The only limitation that occurs to me is that a job can only depend upon a job that comes earlier in the list but I can’t think of a situation in which that would matter. 

   \- more complex still

      @j a 
      @j b  
      @j c &p -1, -2
      @j d &p -1
      @j e &p -2
      @j f &p -3
      @j f &p -1, -2, -3

Here a and b have no prereqs but both are prereqs for c. d, e and f each have only c as a prereq. f has d, e and f as prereqs.

Idea: mimic do_rrule to add jobs to list and then mimic finalize_rruleset to combine the jobs.




## Sort and Groupby

View data "rows" are lists of hashes with keys for *doc_id*, *sort*, *groupby* (YrWk or path) and *columns*. 

- Note that sort must give an order consistent with groupby. E.g., sort by datetime will be consistent with groupby "YrWk".

- Note also that groupby "YrWk" obviates the need for creating a hash with YrWk keys.

As below, the list of hashes is sorted by the "sort" key using itemgetter and then grouped by the "YrWk" or "path" key using groupby. 

In [ ]:
import itertools
import operator

data = [
    {'name': 'Alice', 'age': 30},
    {'name': 'Bob', 'age': 30},
    {'name': 'Charlie', 'age': 25},
    {'name': 'David', 'age': 30},
    {'name': 'Eve', 'age': 25}
]

# Sort the data by 'age' using operator.itemgetter
data.sort(key=operator.itemgetter('age'))

# Group by the 'age' key using operator.itemgetter
grouped = itertools.groupby(data, key=operator.itemgetter('age'))

for key, group in grouped:
    print(f"Age: {key}")
    for item in group:
        print(f"  {item}")


## AttrDict

A dictionary that supports attribute access, e.g., `d.prop = 'value'` and `print(d.prop)` instead of `d['prop']`.

In [ ]:
class AttrDict(dict):
    def __getattr__(self, name):
        try:
            return self[name]
        except KeyError:
            raise AttributeError(f"No such attribute: {name}")

    def __setattr__(self, name, value):
        self[name] = value

# Example usage
d = AttrDict()
d.prop = 'value'
print(d.prop)  # Output: value
print(d['prop'])  # Output: value


## Unit Tests

In [ ]:
# my_module.py
def add(a, b):
    return a + b

def subtract(a, b):
    return a - b

In [ ]:
# pytest actions
# 1) Discover all files matching test_*.py or *_test.py.
# 2) Run all functions and methods prefixed with test_.

# illustrative pytest "test_*.py" file for my_module.py above
# tests/test_my_module.py
import pytest
from my_module import add, subtract

def test_add():
    assert add(1, 2) == 3
    assert add(-1, 1) == 0
    assert add(0, 0) == 0

def test_subtract():
    assert subtract(2, 1) == 1
    assert subtract(2, 2) == 0
    assert subtract(0, 1) == -1

if __name__ == "__main__":
    pytest.main()

## Wrap and Unwrap

This is an amazing helper. Handles wrapping and unwrapping "entry/details" strings while preserving indentations and newlines, setting subsequent indents for outlines, @ and & entries and avoiding breaks between @keys and their values.

Unwrapped text:

```
This is a long string that we want to wrap and then unwrap. We will use the textwrap module to handle the wrapping.
    This is a new paragraph that should also be wrapped.

And this after a blank line. We will use the textwrap module to handle the wrapping.
    * And another new paragraph that should also be wrapped. Now is the time for all good men to come to the aid of their country.
    * And another new paragraph that should also be wrapped. Now is the time for all good men to come to the aid of their country.
+ This is a list item.
- with a sublist item that probably should be wrapped.
* Thanksgiving @s 2010/11/26 @r y &M 11 &w 4TH @d This is another special paragraph. When to the sessions of sweet, silent thought, I summon up remembrances of things past.
* Another list item.
```

Wrapped text (at 40 characters width):

```
This is a long string that we want to
  wrap and then unwrap. We will use the
  textwrap module to handle the
  wrapping.
​    This is a new paragraph that should
      also be wrapped.
​
​And this after a blank line. We will use
  the textwrap module to handle the
  wrapping.
​    * And another new paragraph that
      should also be wrapped. Now is the
      time for all good men to come to
      the aid of their country.
​    * And another new paragraph that
      should also be wrapped. Now is the
      time for all good men to come to
      the aid of their country.
​+ This is a list item.
​  - with a sublist item that probably
    should be wrapped.
​* Thanksgiving @s 2010/11/26 @r y &M 11
  &w 4TH @d This is another special
  paragraph. When to the sessions of
  sweet, silent thought, I summon up
  remembrances of things past.
​* Another list item.
```

In [156]:
import textwrap
import shutil
import re

# Non-printing character
NON_PRINTING_CHAR = '\u200B'
# Placeholder for spaces within special tokens
PLACEHOLDER = '\u00A0'
# Placeholder for hyphens to prevent word breaks
NON_BREAKING_HYPHEN = '\u2011'

def wrap(text: str, indent: int = 3, width: int = shutil.get_terminal_size()[0] - 3):
    # Preprocess to replace spaces within specific "@\S" patterns with PLACEHOLDER
    text = preprocess_text(text)

    # Split text into paragraphs
    paragraphs = text.split('\n')

    # Wrap each paragraph
    wrapped_paragraphs = []
    for para in paragraphs:
        leading_whitespace = re.match(r'^\s*', para).group()
        initial_indent = leading_whitespace

        # Determine subsequent_indent based on the first non-whitespace character
        stripped_para = para.lstrip()
        if stripped_para.startswith(('+', '-', '*', '%', '!', '~')):
            subsequent_indent = initial_indent + ' ' * 2
        elif stripped_para.startswith(('@', '&')):
            subsequent_indent = initial_indent + ' ' * 3
        else:
            subsequent_indent = initial_indent + ' ' * indent

        wrapped = textwrap.fill(
            para,
            initial_indent='',
            subsequent_indent=subsequent_indent,
            width=width)
        wrapped_paragraphs.append(wrapped)

    # Join paragraphs with newline followed by non-printing character
    wrapped_text = ('\n' + NON_PRINTING_CHAR).join(wrapped_paragraphs)

    # Postprocess to replace PLACEHOLDER and NON_BREAKING_HYPHEN back with spaces and hyphens
    wrapped_text = postprocess_text(wrapped_text)

    return wrapped_text

def preprocess_text(text):
    # Regex to find "@\S" patterns and replace spaces within the pattern with PLACEHOLDER
    text = re.sub(r'(@\S+\s\S+)', lambda m: m.group(0).replace(' ', PLACEHOLDER), text)
    # Replace hyphens within words with NON_BREAKING_HYPHEN
    text = re.sub(r'(\S)-(\S)', lambda m: m.group(1) + NON_BREAKING_HYPHEN + m.group(2), text)
    return text

def postprocess_text(text):
    text = text.replace(PLACEHOLDER, ' ')
    text = text.replace(NON_BREAKING_HYPHEN, '-')
    return text

def unwrap(wrapped_text):
    # Split wrapped text into paragraphs
    paragraphs = wrapped_text.split('\n' + NON_PRINTING_CHAR)

    # Replace newlines followed by spaces in each paragraph with a single space
    unwrapped_paragraphs = []
    for para in paragraphs:
        unwrapped = re.sub(r'\n\s*', ' ', para)
        unwrapped_paragraphs.append(unwrapped)

    # Join paragraphs with original newlines
    unwrapped_text = '\n'.join(unwrapped_paragraphs)

    return unwrapped_text

def compare_strings_unified(str1, str2):
    if str1 == str2:
        return True, ''
    # Strip trailing whitespace from each line
    lines1 = [line.rstrip() for line in str1.splitlines()]
    lines2 = [line.rstrip() for line in str2.splitlines()]
    num_lines = max(len(lines1), len(lines2))

    same = True
    changed_lines = []
    for i in range(num_lines):
        if i < len(lines1) and i < len(lines2):
            if lines1[i] != lines2[i]:
                changed_lines.append((i, lines1[i], lines2[i]))
                same = False
        elif i < len(lines1):
            changed_lines.append((i, lines1[i], ''))
            same = False
        else:
            changed_lines.append((i, '', lines2[i]))
            same = False
    if same:
        return True, ''

    # Generate the diff with stripped lines
    diff = list(difflib.unified_diff(lines1, lines2, lineterm=''))

    return False, '\n'.join([f"line {x[0]}:\n  {x[1]}\n  {x[2]}" for x in changed_lines]) + '\n' + '\n'.join(diff)


# Example usage
original_text1 = """\
This is a long string that we want to wrap and then unwrap. We will use the textwrap module to handle the wrapping.
    This is a new paragraph that should also be wrapped.

And this after a blank line. We will use the textwrap module to handle the wrapping."""
original_text1 += "\n    * And another new paragraph that should also be wrapped. Now is the time for all good men to come to the aid of their country." * 2
original_text1 += "\n+ This is a list item.\n  - with a sublist item that probably should be wrapped."
original_text1 += "\n* Thanksgiving @s 2010/11/26\n@r y &M 11 &w 4TH\n@d This is another special paragraph. When to the sessions of sweet, silent thought, I summon up remembrances of things past. "
original_text1 += "\n* Another list item."

original_text2 = """\
% When to the sessions - Shakespeare #30
@c dag @i personal/quotations
@d When to the sessions of sweet silent thought
   I summon up remembrance of things past,
   I sigh the lack of many a thing I sought,
   And with old woes new wail my dear time's waste:
   Then can I drown an eye, unused to flow,
   For precious friends hid in death's dateless night,
   And weep afresh love's long since cancelled woe,
   And moan the expense of many a vanished sight:
   Then can I grieve at grievances foregone,
   And heavily from woe to woe tell o'er
   The sad account of fore-bemoaned moan,
   Which I new pay as if not paid before.
     But if the while I think on thee, dear friend,
     All losses are restored and sorrows end.
"""

# Example usage
original_text3 = "* Thanksgiving @s 2010/11/26 @r y &M 11 &w 4TH. @d Use non-breaking hyphen in hyphenated-words."

original_text = original_text1 + '\n\n' + original_text2 + '\n\n' + original_text3
print("Original text:")
print(original_text)

width = 30

# Wrap the text
wrapped_text = wrap(original_text, indent=2, width=width)
print("Wrapped text:")
print(wrapped_text)

# Unwrap the text
unwrapped_text = unwrap(wrapped_text)
print("\nUnwrapped text:")
print(unwrapped_text)

# Check if the original and unwrapped texts are the same
same, diff = compare_strings_unified(original_text, unwrapped_text)
print("\nOriginal and unwrapped texts are the same:", same)

if not same:
    print("diff:")
    print(diff)


Original text:
This is a long string that we want to wrap and then unwrap. We will use the textwrap module to handle the wrapping.
    This is a new paragraph that should also be wrapped.

And this after a blank line. We will use the textwrap module to handle the wrapping.
    * And another new paragraph that should also be wrapped. Now is the time for all good men to come to the aid of their country.
    * And another new paragraph that should also be wrapped. Now is the time for all good men to come to the aid of their country.
+ This is a list item.
  - with a sublist item that probably should be wrapped.
* Thanksgiving @s 2010/11/26
@r y &M 11 &w 4TH
@d This is another special paragraph. When to the sessions of sweet, silent thought, I summon up remembrances of things past. 
* Another list item.

% When to the sessions - Shakespeare #30
@c dag @i personal/quotations
@d When to the sessions of sweet silent thought
   I summon up remembrance of things past,
   I sigh the lack of many

## ZODB Caching

In [ ]:
from persistent import Persistent

class Item(Persistent):
    def __init__(self, doc_id, name, item_type, item_date=None, completion_date=None):
        self.doc_id = doc_id
        self.item_type = item_type
        self.name = name
        self.item_date = item_date
        self.completion_date = completion_date
        self.is_completed = (
            self.item_type == 'task' and
            self.completion_date is not None
        )

    def get_views_and_rows(self):
        views_and_rows = {}

        # Example logic for adding to agenda view if there is an event date
        if self.item_date:
            YrWk = self.item_date.isocalendar()[:2]
        elif self.completion_date:
            YrWk = self.completion_date.isocalendar()[:2]
        else:
            YrWk = None

        if self.item_type == 'event':
            if self.item_date:
                views_and_rows[(YrWk, 'agenda')] = [(self.item_date, self.name)]

        # Example logic for adding to tasks view if there is a due date and it's not completed
        elif self.item_type == 'task':
            if self.completion_date:
                views_and_rows[(YrWk, 'agenda')] = [(self.completion_date, self.name)]
            elif self.item_date:
                views_and_rows[(YrWk, 'agenda')] = [(self.item_date, self.name)]
            else:
                views_and_rows['tasks'] = [(None, self.name)]

        return views_and_rows


In [ ]:
from ZODB import DB, FileStorage
import transaction
from collections import defaultdict
import os

class ItemManager:
    def __init__(self, db_path=None) -> None:
        if db_path is None:
            db_path = os.path.join(os.getcwd(), "items.fs")
        self.db_path = db_path
        self.storage = FileStorage.FileStorage(self.db_path)
        self.db = DB(self.storage)
        self.connection = self.db.open()
        self.root = self.connection.root()

        # Initialize data structures
        if 'doc_view_data' not in self.root:
            self.root['doc_view_data'] = {}
            self.root['view_doc_data'] = defaultdict(lambda: defaultdict(list))
            self.root['doc_view_contribution'] = defaultdict(set)
            transaction.commit()

        self.doc_view_data = self.root['doc_view_data']
        self.view_doc_data = self.root['view_doc_data']
        self.doc_view_contribution = self.root['doc_view_contribution']
        self.view_cache = {}  # In-memory cache

    def add_or_update_reminder(self, item):
        doc_id = item.doc_id
        new_views_and_rows = item.get_views_and_rows()

        # Invalidate cache for views that will be affected by this doc_id
        self.invalidate_cache_for_doc(doc_id)

        # Update the primary structure
        self.doc_view_data[doc_id] = new_views_and_rows

        # Update the secondary index
        for view, rows in new_views_and_rows.items():
            self.view_doc_data[view][doc_id] = rows
            self.doc_view_contribution[doc_id].add(view)

        transaction.commit()

    def get_view_data(self, view):
        # Check if the view is in the cache
        if view in self.view_cache:
            return self.view_cache[view]

        # Retrieve data for a specific view
        view_data = dict(self.view_doc_data[view])

        # Cache the view data
        self.view_cache[view] = view_data
        return view_data

    def get_reminder_data(self, doc_id):
        # Retrieve data for a specific reminder
        return self.doc_view_data.get(doc_id, {})

    def remove_reminder(self, doc_id):
        # Invalidate cache for views that will be affected by this doc_id
        self.invalidate_cache_for_doc(doc_id)

        # Remove reminder from primary structure
        if doc_id in self.doc_view_data:
            views_and_rows = self.doc_view_data.pop(doc_id)
            # Remove from secondary index
            for view in views_and_rows:
                if doc_id in self.view_doc_data[view]:
                    del self.view_doc_data[view][doc_id]

            # Remove doc_id from contribution tracking
            if doc_id in self.doc_view_contribution:
                del self.doc_view_contribution[doc_id]

        transaction.commit()

    def invalidate_cache_for_doc(self, doc_id):
        # Invalidate cache entries for views affected by this doc_id
        if doc_id in self.doc_view_contribution:
            for view in self.doc_view_contribution[doc_id]:
                if view in self.view_cache:
                    del self.view_cache[view]

    def close(self):
        self.connection.close()
        self.db.close()
        self.storage.close()


In [ ]:
def main():
    db_path = os.path.join(os.getcwd(), "items.fs")
    item_manager = ItemManager(db_path)

    # Add some items
    item1 = Item(1, 'Meeting', 'event', item_date=date(2024, 8, 10))
    item_manager.add_or_update_reminder(item1)

    item2 = Item(2, 'Complete Report', 'task', item_date=date(2024, 8, 12))
    item_manager.add_or_update_reminder(item2)

    item3 = Item(3, 'Submit Assignment', 'task', completion_date=date(2024, 8, 15))
    item_manager.add_or_update_reminder(item3)

    # Query and cache views
    print(item_manager.get_view_data(('2024-32', 'agenda')))  # Query for a specific view
    print(item_manager.get_view_data('tasks'))  # Query for tasks view

    # Invalidate and update cache
    item3.completion_date = date(2024, 8, 16)
    item_manager.add_or_update_reminder(item3)
    print(item_manager.get_view_data(('2024-33', 'agenda')))  # Updated query after invalidation

    item_manager.close()

if __name__ == "__main__":
    main()
